In [1]:
#!/usr/bin/python

#####################################################################
# This script presents SPECTATOR mode. In SPECTATOR mode you play and
# your agent can learn from it.
# Configuration is loaded from "../../examples/config/<SCENARIO_NAME>.cfg" file.
# 
# To see the scenario description go to "../../scenarios/README.md"
# 
#####################################################################
from __future__ import print_function
from vizdoom import *
from time import sleep

game = DoomGame()

In [2]:
# Choose scenario config file you wish to watch.
# Don't load two configs cause the second will overrite the first one.
# Multiple config files are ok but combining these ones doesn't make much sense.

game.load_config("../../examples/config/basic.cfg")

imgs = []
actions=[]
# game.load_config("../../examples/config/deadly_corridor.cfg")
# game.load_config("../../examples/config/deathmatch.cfg")
#game.load_config("../../examples/config/defend_the_center.cfg")
#game.load_config("../../examples/config/defend_the_line.cfg")
#game.load_config("../../examples/config/health_gathering.cfg")
#game.load_config("../../examples/config/my_way_home.cfg")
#game.load_config("../../examples/config/predict_position.cfg")
#game.load_config("../../examples/config/take_cover.cfg")

In [3]:
from tempfile import mkdtemp
import os.path as path
filename = path.join(mkdtemp(), 'newfile.dat')

In [4]:
import numpy as np
images = np.empty([3, 240, 320, 300])
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(3, 240, 320, 300,1000))
ep = 0

In [5]:
# Enables spectator mode, so you can play. Sounds strange but it is agent who is supposed to watch not you.
game.set_window_visible(True)
game.set_mode(Mode.SPECTATOR)

game.init()

episodes = 1

print("")
for i in range(episodes):
    images = np.empty([3, 240, 320, 300])
    print("Episode #" +str(i+1))

    game.new_episode()
    while not game.is_episode_finished():

        s = game.get_state()
        img = s.image_buffer
        misc = s.game_variables

        game.advance_action()
        a = game.get_last_action()
        r = game.get_last_reward()
        
        
        images[:,:,:,s.number - 1] = img 
        
        imgs.append(img)
        actions.append(a)

        print("state #"+str(s.number))
        print("game variables: ", misc)
        print("action:", a)
        print("reward:",r)
        print("=====================")

    print("episode finished!")
    print("total reward:", game.get_total_reward())
    print("************************")
    sleep(2.0)

    ep += 1

game.close()
# fp[:,:,:,:,ep] = images

ViZDoomErrorException: Controlled ViZDoom instance reported error.

In [ ]:
images[:,:,:,146]

In [ ]:
actions[0]

In [ ]:
data = np.arange(12, dtype='float32')
data.resize((3,4))

In [ ]:
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(3, 240, 320, 300,1000))

In [ ]:
 del fp